In [1]:
import pandas as pd
import random

import re
import pickle

In [25]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET

    text = re.sub(r"^https://t.co/[A-Za-z0-9]*\s"," ",text)
    text = re.sub(r"\s+https://t.co/[A-Za-z0-9]*\s"," ",text)
    text = re.sub(r"\s+https://t.co/[A-Za-z0-9]*$"," ",text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]
            

In [56]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-IP.csv")

In [57]:
# For Testing Purpose
ip = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique()
ip_list

array(['193.93.167.87', '205.10.168.66'], dtype=object)

In [58]:
# users which are posting either all positive or negative reviews on different products of same brand

blocked_users = []
size = len(ip_list.tolist())
index = 0
for i in range(size):
    brand_df = ip.get_group(ip_list[i])
    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products<=2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(ip_list[i])
            break
    if(isSameSentiment):
        break;
    index +=k

In [60]:
blocked_users

['193.93.167.87']

In [61]:
dataset.set_index("IP Address",inplace=True)

In [62]:
for ip in blocked_users:
    dataset.drop(ip,inplace=True)

In [63]:
dataset.reset_index()

,IP Address,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,205.10.168.66,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0,0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31
1,205.10.168.66,US,7846966,RY8DDL22YG4R5,B00KA6CCVY,563475445,Aweek® Air Acoustic Earpiece Headset for Baofe...,Mobile_Electronics,3,0,1,N,Y,"It appears to be good, but I'm still waiting f...","It appears to be good, but I'm still waiting f...",2015-08-31
2,205.10.168.66,US,21299354,R2AT2426ZHFUHH,B00MJCDPM2,563475445,Sentey LS-4460 B-Trek S8 Bluetooth Portable St...,Mobile_Electronics,3,0,1,N,Y,Didn't love the first one,"First one arrived as a brick. Wouldn't work, ...",2015-08-31
3,205.10.168.66,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,563475445,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31


In [ ]:
#dataset.to_csv("review3.csv")